In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
match       = pd.read_csv("match_cleaned.csv")
match_30    = pd.read_csv("match_cleaned_drop30.csv")
player_att  = pd.read_csv("player_att_with_endtime.csv")
team_att    = pd.read_csv("team_att_with_endtime.csv")
p_discrete = ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']
unique_vals = { 'preferred_foot': ['right', 'left'],
                'attacking_work_rate': ['medium', 'high', 'low'],
                'defensive_work_rate': ['medium', 'high', 'low']  }
disc_labs = [ lab + '_' + str(val) for lab in p_discrete for val in unique_vals[lab] if val == val ]
disc_labs = [ 'h_' + lab for lab in disc_labs ] + [ 'a_' + lab for lab in disc_labs ]
for lab in disc_labs:
    match[lab] = ''

In [3]:
def calc_disc(row):
    date = row['date']
    h_players = [player for player in row.iloc[10:21] if player == player]
    h = player_att[player_att['player_id'].isin(h_players)]
    h = h[(h['date'] < date) & ((h['next_date'] != h['next_date']) | (h['next_date'] > date))]
    h_out = []
    for lab in p_discrete:
        h_temp = h[lab].value_counts(normalize=True)*100
        h_out.extend([ h_temp[val] if (val in h_temp.index) else np.nan for val in unique_vals[lab] if val == val ])

    a_players = [player for player in row.iloc[21:32] if player == player]
    a = player_att[player_att['player_id'].isin(a_players)]
    a = a[(a['date'] < date) & ((a['next_date'] != a['next_date']) | (a['next_date'] > date))][p_discrete]
    a_out = []
    for lab in p_discrete:
        a_temp = a[lab].value_counts(normalize=True)*100
        a_out.extend([ a_temp[val] if (val in a_temp.index) else np.nan for val in unique_vals[lab] if val == val ])
    return h_out + a_out

In [4]:
match[disc_labs] = match.apply(calc_disc, axis=1, result_type='expand')
match.to_csv("match_w_discrete_cleaned.csv")
match_30[disc_labs] = match_30.apply(calc_disc, axis=1, result_type='expand')
match_30.to_csv("match_w_discrete_cleaned_drop30.csv")